In [1]:
import os
import cv2

In [3]:
# 初始化变量
roi = []
area = []
def draw_rectangle(event, x, y, flags, param):
    global roi, img_temp

    if event == cv2.EVENT_LBUTTONDOWN:
        roi = [(x, y)]  # 记录左上角坐标

    elif event == cv2.EVENT_LBUTTONUP:
        roi.append((x, y))  # 记录右下角坐标
        cv2.rectangle(img_temp, roi[0], roi[1], (0, 255, 0), 2)  # 画出矩形
        cv2.imshow("Image", img_temp)

# 读取图片
img = cv2.imread('train_jpg/0/Video_20240806182752758_frame_26.jpg')  # 修改为你的图片路径
img_temp = img.copy()

cv2.namedWindow("Image")
cv2.setMouseCallback("Image", draw_rectangle)

while True:
    cv2.imshow("Image", img_temp)
    key = cv2.waitKey(1) & 0xFF

    # 按下 's' 键保存ROI并退出
    if key == ord('s'):
        if len(roi) == 2:
            x1, y1 = roi[0]
            x2, y2 = roi[1]
            x, y = min(x1, x2), min(y1, y2)
            w, h = abs(x2 - x1), abs(y2 - y1)
            print(f"ROI: X={x}, Y={y}, W={w}, H={h}")
            area.append([w,h])
        break

    # 按下 'q' 键退出
    if key == ord('q'):
        break

cv2.destroyAllWindows()
print(area)

ROI: X=875, Y=167, W=91, H=459
[[91, 459]]


In [ ]:
def frame_diff_process(frames_folder, diff_folder):
    # 如果差分图像文件夹不存在，则创建
    os.makedirs(diff_folder, exist_ok= True)

    # 获取所有帧的文件名并排序
    frame_files = sorted([f for f in os.listdir(frames_folder) if f.endswith('.jpg')])

    # 读取第一帧作为初始帧
    previous_frame = cv2.imread(os.path.join(frames_folder, frame_files[0]), cv2.IMREAD_GRAYSCALE)

    for i in range(1, len(frame_files)):
        # 读取下一帧
        current_frame = cv2.imread(os.path.join(frames_folder, frame_files[i]), cv2.IMREAD_GRAYSCALE)

        # 计算当前帧和前一帧之间的差异
        frame_diff = cv2.absdiff(current_frame, previous_frame)
        
        # 保存差异图像
        diff_path = os.path.join(diff_folder, f"diff_{i:04d}.jpg")
        cv2.imwrite(diff_path, frame_diff)

        # 更新前一帧
        previous_frame = current_frame

    print('帧差分图像已保存。')

In [ ]:
def analyze_motion(directory):
    label_counts = {}

    for label in os.listdir(directory):
        label_path = os.path.join(directory, label)
        if os.path.isdir(label_path):
            images = [os.path.join(label_path, f) for f in os.listdir(label_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
            images.sort()  # 确保图片是按顺序处理
            total_objects = 0

            for image_path in images:
                frame = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                if frame is None:
                    continue
                
                _, thresh = cv2.threshold(frame, 25, 255, cv2.THRESH_BINARY)
                contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                total_objects += len(contours)  # 累加当前标签下的运动物体数量

            label_counts[label] = total_objects

    # 绘制条形图
    labels = list(label_counts.keys())
    counts = [label_counts[label] for label in labels]

    plt.figure(figsize=(10, 5))
    plt.bar(labels, counts, color='blue')
    plt.xlabel('Label')
    plt.ylabel('Number of Moving Objects')
    plt.title('Number of Moving Objects per Label')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# 使用方法
base_directory = 'train_jpg_diff_video'  # 更改为你的一级目录路径
analyze_motion(base_directory)